In [2]:
import xgboost as xgb
import pandas as pd
import numpy as np
import joblib
import time
import pyarrow.parquet as pq
import warnings

# Ignora avisos chatos de versão
warnings.filterwarnings("ignore")

# =============================================================================
# 1. CARREGAR A INTELIGÊNCIA
# =============================================================================
MODELO_FILE = r"E:\Estudos_Cybersecurity\Datasets_Cybersecurity\modelo_xgboost.json"
LABEL_ENCODER_FILE = r"E:\Estudos_Cybersecurity\Datasets_Cybersecurity\label_encoder.joblib"
DADOS_LIMPOS = r"E:\Estudos_Cybersecurity\Datasets_Cybersecurity\dataset_limpo.parquet"

print(">>> INICIANDO SISTEMA DE DEFESA (SIMULAÇÃO V2 - LEVE) <<<")

# Carrega o modelo
model = xgb.XGBClassifier()
model.load_model(MODELO_FILE)
print("✅ Cérebro Neural Carregado.")

# Carrega o tradutor de nomes
le = joblib.load(LABEL_ENCODER_FILE)
print("✅ Base de Conhecimento de Ameaças Carregada.")

# =============================================================================
# 2. PREPARAR DADOS DE SIMULAÇÃO (Estratégia Low RAM)
# =============================================================================
print("⏳ Capturando pacotes da rede (Modo Leitura Parcial)...")

try:
    # TRUQUE: Em vez de ler tudo, lemos apenas o primeiro "bloco" do arquivo
    arquivo_parquet = pq.ParquetFile(DADOS_LIMPOS)
    # Lê apenas o primeiro grupo de linhas (geralmente umas 100k linhas)
    df_chunk = arquivo_parquet.read_row_group(0).to_pandas()
    
    # Agora sim, pegamos 50 amostras desse pedaço menor
    amostra_simulacao = df_chunk.sample(50, random_state=42)
    
    # Libera memória do chunk
    del df_chunk
    
except Exception as e:
    print(f"❌ Erro ao ler dados: {e}")
    exit()

# Separa o gabarito (Label) dos dados de entrada (X)
y_real_codigos = le.transform(amostra_simulacao['Label']) # Codifica para comparar números
nomes_reais = amostra_simulacao['Label'].values
X_input = amostra_simulacao.drop(columns=['Label'])

print(f"📡 Capturados {len(X_input)} pacotes. Iniciando análise em tempo real...\n")

# Cabeçalho do Monitor
print("-" * 85)
print(f"{'TIMESTAMP':<15} | {'PREVISÃO IA':<25} | {'STATUS':<20} | {'REALIDADE'}")
print("-" * 85)

# =============================================================================
# 3. LOOP DE DETECÇÃO (SIMULAÇÃO REAL-TIME)
# =============================================================================
contadores = {"Bloqueados": 0, "Permitidos": 0, "Erros": 0}

# Cores para o terminal (Simulação estilo Matrix)
VERDE = "\033[92m"
VERMELHO = "\033[91m"
AMARELO = "\033[93m"
RESET = "\033[0m"

for i in range(len(X_input)):
    # 1. Pega um único pacote
    pacote = X_input.iloc[[i]]
    realidade = nomes_reais[i]
    
    # 2. O Modelo Analisa
    pred_cod = model.predict(pacote)[0]
    pred_nome = le.inverse_transform([pred_cod])[0]
    
    # 3. Simula tempo de processamento
    time.sleep(0.05) # Rápido
    timestamp = pd.Timestamp.now().strftime('%H:%M:%S.%f')[:-3]
    
    # 4. Lógica do Firewall
    if pred_nome == "BENIGN":
        status_msg = "✅ PERMITIDO"
        cor_linha = VERDE
        contadores["Permitidos"] += 1
    else:
        status_msg = f"🚨 BLOQUEADO"
        cor_linha = VERMELHO
        contadores["Bloqueados"] += 1
        
    # Verifica se a IA acertou (Auditoria em tempo real)
    check = "MATCH" if pred_nome == realidade else "MISS"
    
    # Print formatado
    # Se for ataque, destaca a linha inteira em vermelho
    if pred_nome != "BENIGN":
        print(f"{cor_linha}{timestamp:<15} | {pred_nome:<25} | {status_msg:<20} | (Era: {realidade}){RESET}")
    else:
        print(f"{timestamp:<15} | {pred_nome:<25} | {status_msg:<20} | (Era: {realidade})")

print("-" * 85)
print(f"RESUMO DA SESSÃO:")
print(f"{VERDE}🟢 Tráfego Legítimo Liberado: {contadores['Permitidos']}{RESET}")
print(f"{VERMELHO}🔴 Ameaças Bloqueadas:      {contadores['Bloqueados']}{RESET}")

>>> INICIANDO SISTEMA DE DEFESA (SIMULAÇÃO V2 - LEVE) <<<
✅ Cérebro Neural Carregado.
✅ Base de Conhecimento de Ameaças Carregada.
⏳ Capturando pacotes da rede (Modo Leitura Parcial)...
📡 Capturados 50 pacotes. Iniciando análise em tempo real...

-------------------------------------------------------------------------------------
TIMESTAMP       | PREVISÃO IA               | STATUS               | REALIDADE
-------------------------------------------------------------------------------------
22:42:11.780    | UDP                       | 🚨 BLOQUEADO          | (Era: DrDoS_SSDP)
22:42:11.851    | DrDoS_NetBIOS             | 🚨 BLOQUEADO          | (Era: DrDoS_NetBIOS)
22:42:11.915    | DrDoS_NetBIOS             | 🚨 BLOQUEADO          | (Era: DrDoS_NetBIOS)
22:42:11.981    | DrDoS_NetBIOS             | 🚨 BLOQUEADO          | (Era: DrDoS_NetBIOS)
22:42:12.047    | DrDoS_NTP                 | 🚨 BLOQUEADO          | (Era: DrDoS_NTP)
22:42:12.115    | BENIGN                    | ✅ PERMITIDO  